In [1]:
from gensim.models import Word2Vec
from Bio import SeqIO


### Preprocessing

In [21]:

# fasta_file = "data/ecoli.fasta"
fasta_file = "data/chr10.fa"

sequence = ""
for record in SeqIO.parse(fasta_file, "fasta"):
    sequence += str(record.seq).upper()



In [3]:

def get_kmers(seq, k):
    return [seq[i:i+k] for i in range(len(seq)-k+1)]

kmers = get_kmers(sequence, k=6)
print(len(sequence))
print(len(kmers))

4641652
4641647


### Prepare the model

In [4]:
window_size = 5
sentences = []
for i in range(len(kmers)-window_size):
    sentences.append(kmers[i:i+window_size])


model = Word2Vec(sentences, vector_size=100, window=5, sg=1, min_count=1, workers=4)

# Save model
model.save("dna2vec.model")

# --- Step 5: Example: get vector for a k-mer ---
vector = model.wv["ATGCGA"]
print(vector[:5])

[-0.5365001   0.11623012 -0.09589631 -0.2818168   0.54227304]


### Evaluation Metric

In [5]:
from Bio import pairwise2
from scipy.spatial.distance import cosine
import numpy as np

def embed_sequence(seq, k=6):
    kmers = get_kmers(seq, k)
    vecs = [model.wv[kmer] for kmer in kmers if kmer in model.wv]
    return np.mean(vecs, axis=0) if vecs else np.zeros(model.vector_size)

# Example pairs
seq1 = sequence[1000:1030]
seq2 = sequence[2000:2030]

# Embedding cosine similarity
vec1, vec2 = embed_sequence(seq1), embed_sequence(seq2)
cos_sim = 1 - cosine(vec1, vec2)

# Alignment score (Needleman-Wunsch)
alignments = pairwise2.align.globalxx(seq1, seq2, score_only=True)
alignment_score = alignments

print("Cosine similarity:", cos_sim)
print("Alignment score:", alignment_score)


Cosine similarity: 0.6350506
Alignment score: 19.0


c:\Users\Rifat1493\miniconda3\envs\kroger\Lib\site-packages\Bio\pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


### Calculate correlation

In [ ]:
import random
pairs = [(random.randint(0, len(sequence)-6), random.randint(0, len(sequence)-6)) for _ in range(1000)]

cos_sims, aln_scores = [], []
for i, j in pairs:
    s1, s2 = sequence[i:i+6], sequence[j:j+6]
    vec1, vec2 = embed_sequence(s1), embed_sequence(s2)
    cos_sims.append(1 - cosine(vec1, vec2))
    aln_scores.append(pairwise2.align.globalxx(s1, s2, score_only=True))

# Correlation
from scipy.stats import pearsonr
r, p = pearsonr(cos_sims, aln_scores)
print("Pearson correlation between cosine sim and alignment:", r)


Pearson correlation between cosine sim and alignment: 0.5608535563133649


In [13]:
print(sequence[:35])

AGCTTTTCATTCTGACTGCAACGGGCAATATGTCT


In [15]:
s1

'CGCCAA'

In [8]:
pairs

[(308343, 3050428),
 (10460, 3687838),
 (2554184, 3097912),
 (2298610, 3741049),
 (3536926, 3234216),
 (4155762, 1056822),
 (151219, 2526834),
 (3349565, 1208968),
 (314862, 3467845),
 (3448620, 1635601),
 (35190, 3806828),
 (1017733, 501347),
 (1350796, 403004),
 (933420, 4137474),
 (4490069, 1622901),
 (356024, 624010),
 (1844467, 1374594),
 (668773, 185416),
 (3957253, 3691188),
 (525811, 3650749),
 (2177844, 3500415),
 (4462417, 3251604),
 (1129834, 695460),
 (234315, 2096850),
 (1183036, 3713983),
 (338088, 1436667),
 (4590318, 4375561),
 (4198118, 3650057),
 (1452438, 269611),
 (934274, 2434817),
 (1315976, 3428182),
 (1112428, 4025899),
 (3040453, 2329087),
 (1623409, 1005101),
 (3726029, 3038332),
 (1044062, 1560718),
 (4537871, 2484644),
 (412434, 1381760),
 (2773975, 1449234),
 (4079483, 265284),
 (3244293, 2701884),
 (342189, 3840400),
 (2424187, 2497529),
 (2682681, 4314159),
 (574215, 1292412),
 (299020, 3954862),
 (61619, 2797284),
 (1945346, 3809776),
 (4197772, 2451043)

In [24]:
# Check presence
has_dash = '-' in sequence
has_X = 'X' in sequence

print("Has '-' gap:", has_dash)
print("Has 'X':", has_X)


Has '-' gap: False
Has 'X': False


In [26]:
has_non_acgt = any(c not in "ACGT" for c in sequence)
total_non_acgt = sum(1 for c in sequence if c not in "ACGT")
unique_non_acgt = sorted({c for c in sequence if c not in "ACGT"})

print("Has non-ACGT:", has_non_acgt)
print("Total non-ACGT:", total_non_acgt)
print("Unique non-ACGT:", unique_non_acgt)

Has non-ACGT: True
Total non-ACGT: 534460
Unique non-ACGT: ['N']
